In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

In [2]:
class FDN(nn.Module):
    def __init__(self, backbone, num_latent):
        super(FDN, self).__init__()
        self.backbone = backbone
        self.num_latent = num_latent
        self.linears = nn.ModuleList([nn.Linear(512, 128, bias=False) for i in range(num_latent)])
        
    def forward(self, x):
        # out (1, 512)
        x = self.backbone(x)
        x = torch.squeeze(x)
        print(x.shape)
        # x = x.view(x.shape[0], 1, x.shape[1])
        tmp = []
        for i in range(self.num_latent):                                                     
            tmp.append(torch.relu(self.linears[i](x)))
        res = torch.cat(tmp, dim=1).view(-1, self.num_latent, 128)
        return res

In [21]:
class Intra_RM(nn.Module):
    def __init__(self, num_latent):
        super(Intra_RM, self).__init__()
        self.num_latent = num_latent
        self.linears = nn.ModuleList([nn.Linear(128, 128, bias=False) for i in range(num_latent)])
        
    def forward(self, x):
        res1 = []
        for i in range(self.num_latent):
            res1.append(torch.relu(self.linears[i](x)))
        res2 = torch.cat(res1, dim=1).view(-1, self.num_latent, 128)
        return res2
            

In [ ]:
def forward(self, I):                                                                                                                                                                                                               
  1 ▏   ▏   weights = []                                                                                                                                                                                                                    
  2 ▏   ▏   res = []                                                                                                                                                                                                                        
  3 ▏   ▏   for i in range(self.M):                                                                                                                                                                                                         
  4 ▏   ▏   ▏   x = I[:, i, :]                                                                                                                                                                                                              
  5 ▏   ▏   ▏   alpha = torch.sigmoid(self.linears[i](x))                                                                                                                                                                                   
  6 ▏   ▏   ▏   importance = torch.norm(alpha, p=1, dim=-1, keepdim=True)                                                                                                                                                                   
  7 ▏   ▏   ▏   temp = x * importance                                                                                                                                                                                                       
  8 ▏   ▏   ▏   res.append(temp)                                                                                                                                                                                                            
  9 ▏   ▏   ▏   weights.append(importance)                                                                                                                                                                                                  
 10 ▏   ▏   res = torch.cat(res, dim=1).view((-1, self.M, 128))                                                                                                                                                                             
 11 ▏   ▏   weights = torch.cat(weights, dim=1)                                                                                                                                                                                             
 12 ▏   ▏   return res, weights 

In [22]:
model = models.resnet18(pretrained=True)
new_model = torch.nn.Sequential(*(list(model.children())[:-1]))

In [23]:
myFDRL = FDN(new_model, 10)

In [24]:
img = torch.rand(2, 3, 224, 224)
res1 = myFDRL(img)
res1.shape

torch.Size([2, 512])


torch.Size([2, 10, 128])

In [25]:
myIntra = Intra_RM(10)

In [26]:
res2 = myIntra(res1)
res2.shape

torch.Size([20, 10, 128])